# HAR Binary Classifier - Sitting vs Standing

This classifier distinguishes between sitting and standing poses using the MPII Human Pose Dataset.

It allows users to upload their own photos and predicts whether the image contains someone either sitting or standing, and providing a confidence score.

**The code:**

This block checks if you're in Google Colab, installs the required fastbook library, and sets up the basic environment for running fastai.

In [ ]:
# 1. Setup and Imports
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

This imports all the necessary functions and classes from fastbook and fastai's vision module that you'll need for image processing and deep learning

In [ ]:
# 2. Main Imports
from fastbook import *
from fastai.vision.all import *

This downloads the MPII dataset...

In [ ]:
# 2. Data Setup
path = untar_data(URLs.MPII)/'images'  # Changed from PETS to MPII dataset

Defines a function that labels images as sitting returning True if the image shows a sitting pose based on filename metadata. DOUBLE CHEKC MPII LABELLING SCHEME.

In [ ]:
# 3. Label Function
def is_sitting(x):
    """Returns True if the image shows a sitting pose based on filename metadata"""
    # You'll need to adjust this based on MPII's labeling scheme
    return 'sit' in str(x).lower()

added>>>

In [ ]:
# 4. Data Loading with Augmentation
dls = ImageDataLoaders.from_name_func(
    path,
    get_image_files(path),
    valid_pct=0.2,
    seed=42,
    label_func=is_sitting,
    item_tfms=Resize(224),
    batch_tfms=[
        *aug_transforms(flip_vert=False),  # No vertical flips for human poses
        Normalize.from_stats(*imagenet_stats)
    ]
)

added>>>

In [ ]:
# 5. Model Creation & Training
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(3)  # Training for more epochs due to complexity

added>>>

In [ ]:
# 6. File Upload Interface
uploader = widgets.FileUpload()
uploader

added>>>

In [ ]:
# 7. Prediction
def predict_pose(img_data):
    img = PILImage.create(img_data)
    is_sitting,_,probs = learn.predict(img)
    print(f"Is this person sitting?: {is_sitting}")
    print(f"Probability of sitting: {probs[1].item():.6f}")
    return is_sitting, probs

added>>>

In [ ]:
# Example usage
if uploader.data:
    predict_pose(uploader.data[0])

-----

---

This creates the data loaders that will feed images to the model, splitting data into training (80%) and validation (20%) sets, and resizing all images to 224x224 pixels.

In [ ]:
# 4. Data Loading
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))

This creates a vision model using ResNet34 architecture and fine-tunes it for one epoch on your dataset.

In [ ]:
# 5. Model Creation & Training
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

This block creates a file upload widget

In [ ]:
# 6. File Upload (separate cell)
uploader = widgets.FileUpload()
uploader

 Then handle making predictions on uploaded images.

In [ ]:
# 7. Prediction (separate cell)
img = PILImage.create(uploader.data[0])
is_cat,_,probs = learn.predict(img)
print(f"Is this a cat?: {is_cat}.")
print(f"Probability it's a cat: {probs[1].item():.6f}")

Is this a cat?: False.
Probability it's a cat: 0.062023
